## --- 最大值对称法

### 定点计算和浮点数计算测试性能

In [17]:
import sys
import time
import numpy as np

# 随机生成一些浮点数 (float32)
data_float32 = np.random.randn(10).astype("float32")

# 量化上下限
Qmin = 0
Qmax = 255

# 计算缩放因子
S = (data_float32.max() - data_float32.min()) / (Qmax - Qmin)

# 计算零点
Z = Qmax - data_float32.max() / S

# 将浮点数据 (float32)量化为定点数据 (UInt8)
data_uint8 = np.round(data_float32 / S + Z).astype("uint8")

# 将定点数据 (uint8 ) 反量化为浮点数据 (float32)
data_float32_ = ((data_uint8 - Z) * S).astype("float32")

####### 误差计算
# 使用均方误差计算误差
mse = ((data_float32 - data_float32_)**2).mean()

print(f"原始数据: {data_float32}")
print(f"反量化后的数据: {data_float32_}")
print(f"量化后的数据: {data_uint8}")
print(f"原始数据和反量化数据的均方误差: {mse}")

原始数据: [-0.5124247   0.54156023  0.16993663 -0.28651538 -0.4935727   1.1646749
  1.0023637  -1.0460062   0.8246637  -0.51910216]
反量化后的数据: [-0.5085072   0.5404826   0.16770111 -0.28310442 -0.4911685   1.1646749
  0.9999575  -1.0460061   0.82657075 -0.5171765 ]
量化后的数据: [ 62 183 140  88  64 255 236   0 216  61]
原始数据和反量化数据的均方误差: 5.205538855079794e-06


In [18]:
############### 内存对比

print(f"\n内存对比")

# 空数组的内存占用
empty_size = sys.getsizeof(np.array([]))
print(f"空数据数据内存占用: {empty_size} bytes")



float32_size = sys.getsizeof(data_float32) - empty_size
uint8_size = sys.getsizeof(data_uint8) - empty_size

print(f"原始数据内存占用: {float32_size} bytes")
print(f"量化后的数据内存占用: {uint8_size} bytes")
print(f"量化数据和原始数据内存占比: {uint8_size/float32_size} ")


内存对比
空数据数据内存占用: 104 bytes
原始数据内存占用: 40 bytes
量化后的数据内存占用: 10 bytes
量化数据和原始数据内存占比: 0.25 


In [21]:
####### 速度占比

print(f"速度占比\n")

repeat_num = 100000

t_s = time.time()
sum_float32 = 0
for i in range(repeat_num):
    sum_float32 += data_float32 * 1 + 1

t_e = time.time()
f32_time = t_e - t_s

t_s = time.time()
sum_uint8 = 0
for i in range(repeat_num):
    sum_uint8 += data_uint8 * 1 + 1
t_e = time.time()
uint8_time = t_e - t_s

print(f"原始数据求和  {repeat_num} 次耗时 {f32_time}")
print(f"量化后数据求和  {repeat_num} 次耗时 {uint8_time}")
print(f"耗时减少的倍数: {uint8_time / f32_time}")

速度占比

原始数据求和  100000 次耗时 0.2520327568054199
量化后数据求和  100000 次耗时 0.1900022029876709
耗时减少的倍数: 0.7538790012695086
